# Causation Corpus classification 
#### Chain of Thoughts Self Consistency

This notebook uses the chain-of-thoughts self consistency prompting technique to classify sentences via LLMs.

## OpenAI Privacy Policy
This notebook uses OpenAI's API, meaning that your data will be sent to the OpenAI servers.

For concerns about how your data will be handled, please read through the Privacy Policy [here](https://openai.com/policies/api-data-usage-policies).

## Notebook Overview

In [ ]:
# prompt set up
# upload the prompt file
# pre - instructions: this is your prompt file.
# post - validate: validate prompt file.
# post - feedback: collapsible/viewable widget for what's uploaded.
# design: uploader, then collapsible pop down of what's uploaded and what's validated.

In [1]:
from causation.utils import openai_apikey_input

openai_apikey_input()

Row
    [0] PasswordInput(name='Enter your OpenAI A..., placeholder='<OpenAI API Key>')
    [1] Markdown(str)

In [2]:
# upload the dataset
# pre - instructions: 
# post - feedback: summary of what's been uploaded. Which column is used for classification.

## Classification

In [3]:
# design rationale:
# controller pane -> this will display all the necessary information (model config + tikdollar, prompt + dataset summary, expected output)

# api-key first then pop down.
# model config, tikdollar, apikey

In [17]:
import panel as pn; pn.extension()

selector = pn.widgets.Select(options=['gpt-3.5-turbo', 'gpt-3.5-turbo-16k'])
# model selection
mconfig = pn.Column("## Model Configuration", 
                    selector,
                    pn.widgets.TooltipIcon(value="This is a simple tooltip by using a string", margin=(-33, -500, 20, -170)),
                    pn.widgets.FloatSlider(name="Top p", start=0.1, end=1.0, step=0.1, value=0.8, tooltips=True),
                    pn.widgets.TooltipIcon(value="This is a simple tooltip by using a string", margin=(-43, -40, 30, -170)),
                    pn.widgets.FloatSlider(name="Temperature", start=0.0, end=2.0, step=0.1, value=1.0, tooltips=False), 
                    pn.widgets.TooltipIcon(value="This is a simple tooltip by using a string", margin=(-43, -120, 50, -170)))
mconfig

Column
    [0] Markdown(str)
    [1] Select(options=['gpt-3.5-turbo', ...], value='gpt-3.5-turbo')
    [2] TooltipIcon(margin=(-33, -500, 20, -170), value='This is a simple t...)
    [3] FloatSlider(name='Top p', start=0.1, value=0.8)
    [4] TooltipIcon(margin=(-43, -40, 30, -170), value='This is a simple t...)
    [5] FloatSlider(end=2.0, name='Temperature', tooltips=False, value=1.0)
    [6] TooltipIcon(margin=(-43, -120, 50, -170), value='This is a simple t...)

In [18]:
config = dict(model=selector.value)
config

{'model': 'gpt-3.5-turbo'}

In [19]:
config

{'model': 'gpt-3.5-turbo'}

In [20]:
import pandas as pd

df = pd.DataFrame()

instructions = """
You are an expert in philosophy, spefically in the domain of biases.
You are trying to identify biases that deterministically or categorically link genes to "traits" or "phenotypes", including obese, obesity, overweight, diabetic, diabetes, heavy, fat, fatness; and also behaviours such as eating, overeating, hunger, hungry, craving, fat storage, weight gain, gaining weight, weight loss, losing weight, exercise, physical activity, burning calories.\
Sentence must be about genes causing a trait, not about a trait affecting a gene.
When genes "do not", "don't", or "didn't" cause or determine or lead to a trait, then no bias. 
"""

html_str = """
<h3>Instructions</h3>
{instructions}
"""

clz_collapsible = pn.Accordion(('class', 
              pn.Column(pn.pane.HTML(html_str.format(instructions=instructions).lstrip()),
              pn.pane.DataFrame(df, index=False, height=200, sizing_mode='stretch_width'))))

In [21]:
from panel.theme import Material
dataset_df = pd.DataFrame([f'query {i}' for i in range(100)], columns=['sentence'])

dataset_pane = pn.pane.DataFrame(dataset_df, index=False, height=400, sizing_mode='stretch_width', justify='left')
dataset_widget = pn.widgets.DataFrame(dataset_df, show_index=False, height=400, sizing_mode='stretch_width', disabled=True, design=Material)

In [22]:
import panel as pn; pn.extension()
import time
tqdm = pn.widgets.Tqdm()

def run_loop(*events, timeout=0.2):
    for i in tqdm(range(0,10), desc="My loop bar", leave=True, colour='#666666'):
        if pn.state._is_pyodide:
            # time.sleep does not work in pyodide
            np.random.random((10**6, 30))  
        else:
            time.sleep(timeout)
tqdm

Tqdm(layout=Column, progress=Progress, text_pane=Str)

In [23]:
tqdm

Tqdm(layout=Column, progress=Progress, text_pane=Str)

In [24]:
run_loop()

In [32]:
from panel.theme import Material
controller = pn.Row(pn.Column(mconfig, pn.widgets.Button(name="Run classification", button_type='primary', design=Material)), 
                    pn.Spacer(width=20), 
                    pn.Tabs(("Prompt", pn.Column(pn.widgets.FileInput(accept=".toml", design=Material), clz_collapsible)), 
                            ("Dataset", pn.Column(pn.widgets.FileInput(accept=".csv,.xlsx", design=Material), dataset_widget)), 
                            ("Classification", None),
                            design=Material), 
                    design=Material)
controller = pn.Column(controller, tqdm, sizing_mode='stretch_both')
controller

Column(sizing_mode='stretch_both')
    [0] Row(design=<class 'panel.theme.materi...)
        [0] Column
            [0] Column
                [0] Markdown(str)
                [1] Select(options=['gpt-3.5-turbo', ...], value='gpt-3.5-turbo')
                [2] TooltipIcon(margin=(-33, -500, 20, -170), value='This is a simple t...)
                [3] FloatSlider(name='Top p', start=0.1, value=0.8)
                [4] TooltipIcon(margin=(-43, -40, 30, -170), value='This is a simple t...)
                [5] FloatSlider(end=2.0, name='Temperature', tooltips=False, value=1.0)
                [6] TooltipIcon(margin=(-43, -120, 50, -170), value='This is a simple t...)
            [1] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Run classification')
        [1] Spacer(width=20)
        [2] Tabs(design=<class 'panel.theme.materi...)
            [0] Column
                [0] FileInput(accept='.toml', design=<class 'panel.theme.materi...)
                [1] Accordion
                    [0] Column
                        [0] HTML(str)
                        [1] DataFrame(DataFrame, height=200, index=False, sizing_mode='stretch_width')
            [1] Column
                [0] FileInput(accept='.csv,.xlsx', design=<class 'panel.theme.materi...)
                [1] DataFrame(design=<class 'panel.theme.materi..., disabled=True, height=400, show_index=False, sizing_mode='stretch_width', value=    sentence
0  ...)
            [2] Str(None, name='Classification')
    [1] Tqdm(layout=Column, max=10, progress=Progress, text='My loop bar: 100% 10/10 [..., text_pane=Str, value=10)

In [106]:
import numpy as np

stream_df = pd.DataFrame(np.random.randn(10, 5), columns=list('ABCDE'))

stream_table = pn.widgets.DataFrame(stream_df, autosize_mode='fit_columns', width=450)
stream_table

DataFrame(autosize_mode='fit_columns', value=          A   ..., width=450)

In [137]:
def stream_data():
    stream_df = pd.DataFrame(np.random.randn(10, 1), columns=list('F'))
    stream_table.stream(stream_df)

pn.state.add_periodic_callback(stream_data, period=1000, count=5)

PeriodicCallback(callback=<function stream_data at 0x163d11cf0>, count=5, counter=0, log=True, name='PeriodicCallback03755', period=1000, running=True, timeout=None)

In [141]:
df['classification'] = None
patch_table = pn.widgets.DataFrame(df, show_index=False, style=)
patch_table

DataFrame(show_index=False, value=              ...)

In [142]:
patch_table.patch({
    'classification': [
        (0, 'hello'),
    ],
})


In [122]:
dial = pn.indicators.Dial(
    name='$ Spent', value=2500, bounds=(0, 3000), format='${value}',
    colors=[(0.2, 'green'), (0.8, 'gold'), (1, 'red')],
    width=150, height=150
)
dial

Dial(bounds=(0, 3000), colors=[(0.2, 'green'), ...], format='${value}', height=150, name='$ Spent', value=2500, width=150)

In [ ]:
# model set up
# model name -> selection widget
# sampling scheme
from llm_experiments import SamplingScheme

sampling_scheme = SamplingScheme(top_p=0.8, temperature=1, presence_penalty=0.0)
n_completions = 3

In [ ]:
# user input: confirmation
# + how many classifications will be run.
# + tikdollar set up
# + expected columns in the output.xlsx.
# + pop up view of the prompt + dataset summary + model config + tikdollar + expected output.xlsx
# + feedback: run -> green -> running -> progress bar.
# run classification
# expensive compute: progressive checkpointing @50 (alert) 
# feedback: progress bar + tikdollar indicator (container: row)
# feedback: completion of classification -> how much is spent -> download xlsx.

In [158]:
import panel as pn; pn.extension()
import time

btn = pn.widgets.Button(name='click me')

def on_click(event):
    time.sleep(1)
    return event
    
def on_click2(event2):
    return None

iobj = pn.bind(on_click, btn)
iobj2 = pn.bind(on_click2, btn)
pn.Row(btn, iobj, pn.widgets.DataFrame(iobj2))

Row
    [0] Button(name='click me')
    [1] ParamFunction(function, _pane=Str, defer_load=False)
    [2] DataFrame()

In [2]:
from causation.widget import DatasetUpload, FileUploadToDir

# uploader = DatasetUpload('sentence', upload_dir='./uploads')
# uploader.widget()
uploader = FileUploadToDir(ext=['.xlsx'], upload_dir='./uploads')
uploader.widget()

Row(design=<class 'panel.theme.materi...)
    [0] FileInput(accept='.xlsx', design=<class 'panel.theme.materi...)
    [1] Markdown(str, design=<class 'panel.theme.materi...)

In [1]:
from causation.widget import DatasetUpload, FileUploadToDir

uploader = DatasetUpload('sentence', upload_dir='./uploads')
uploader.widget()

called
None


Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [0] Row(design=<class 'panel.theme.materi...)
        [0] FileInput(accept='.xlsx,.csv', design=<class 'panel.theme.materi...)
        [1] Markdown(str, design=<class 'panel.theme.materi...)
    [1] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)

In [1]:
from causation.widget import DatasetUpload, FileUploadToDir, CoTPromptUpload

uploader = CoTPromptUpload(upload_dir='./uploads')
uploader.widget()

Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [0] Row(design=<class 'panel.theme.materi...)
        [0] FileInput(accept='.toml', design=<class 'panel.theme.materi...)
        [1] Markdown(str, design=<class 'panel.theme.materi...)
    [1] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)

In [1]:
from causation.widget import DatasetUpload, FileUploadToDir, CoTPromptUpload, ModelConfig

mconfig = ModelConfig()
mconfig.widget()

Column(design=<class 'panel.theme.materi...)
    [0] Markdown(str, design=<class 'panel.theme.materi...)
    [1] Select(design=<class 'panel.theme.materi..., options=['gpt-3.5-turbo', ...], value='gpt-3.5-turbo')
    [2] TooltipIcon(design=<class 'panel.theme.materi..., margin=(-33, -500, 20, -170), value='This is a simple t...)
    [3] FloatSlider(design=<class 'panel.theme.materi..., name='Top p', start=0.1, value=0.8)
    [4] TooltipIcon(design=<class 'panel.theme.materi..., margin=(-43, -40, 30, -170), value='This is a simple t...)
    [5] FloatSlider(design=<class 'panel.theme.materi..., end=2.0, name='Temperature', tooltips=False, value=1.0)
    [6] TooltipIcon(design=<class 'panel.theme.materi..., margin=(-43, -120, 50, -170), value='This is a simple t...)

In [3]:
mconfig.get_config()

Config(model='gpt-3.5-turbo-16k', top_p=0.8, temperature=1.0)

In [1]:
from causation.widget import Controller

Controller(col_sentence='sentence', upload_dir='./uploads').widget()

Row(design=<class 'panel.theme.materi...)
    [0] Column(design=<class 'panel.theme.materi...)
        [0] Column(design=<class 'panel.theme.materi...)
            [0] Markdown(str, design=<class 'panel.theme.materi...)
            [1] Select(design=<class 'panel.theme.materi..., options=['gpt-3.5-turbo', ...], value='gpt-3.5-turbo')
            [2] TooltipIcon(design=<class 'panel.theme.materi..., margin=(-33, -500, 20, -170), value='The GPT model to use.')
            [3] FloatSlider(design=<class 'panel.theme.materi..., name='Top p', start=0.1, value=0.8)
            [4] TooltipIcon(design=<class 'panel.theme.materi..., margin=(-43, -40, 30, -170), value="Makes the answer more foc...)
            [5] FloatSlider(design=<class 'panel.theme.materi..., end=2.0, name='Temperature', tooltips=False, value=1.0)
            [6] TooltipIcon(design=<class 'panel.theme.materi..., margin=(-43, -120, 50, -170), value='Changes how random the an...)
    [1] Spacer(design=<class 'panel.theme.materi..., width=20)
    [2] Tabs(design=<class 'panel.theme.materi...)
        [0] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] Row(design=<class 'panel.theme.materi...)
                [0] FileInput(accept='.toml', design=<class 'panel.theme.materi...)
                [1] Markdown(str, design=<class 'panel.theme.materi...)
            [1] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)
        [1] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] Row(design=<class 'panel.theme.materi...)
                [0] FileInput(accept='.xlsx,.csv', design=<class 'panel.theme.materi...)
                [1] Markdown(str, design=<class 'panel.theme.materi...)
            [1] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)
        [2] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] Row(design=<class 'panel.theme.materi...)
                [0] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Run classification')
                [1] Spacer(design=<class 'panel.theme.materi..., width=20)
                [2] Tqdm(design=<class 'panel.theme.materi..., layout=Column, progress=Progress, text_pane=Str, width=500)
            [1] DataFrame(design=<class 'panel.theme.materi..., value=Empty DataFrame
Columns: [...)